# 시계열 분석

```sql
-- 1.1. 월별 계약 발생 건수 집계
-- 월별 계약 발생 건수
SELECT
    DATE_TRUNC('month', issue_date) AS month,
    COUNT(*) AS contract_issued
FROM contract_basic
GROUP BY DATE_TRUNC('month', issue_date)
ORDER BY month;

-- 1.2. 월별 계약 종료 건수 집계
-- 월별 계약 종료 건수
SELECT
    DATE_TRUNC('month', expiry_date) AS month,
    COUNT(*) AS contract_expired
FROM contract_basic
WHERE expiry_date IS NOT NULL
GROUP BY DATE_TRUNC('month', expiry_date)
ORDER BY month;

-- 1.3. 발생 및 종료 건수를 통합
-- 계약 발생 및 종료 건수 통합
WITH issued AS (
    SELECT
        DATE_TRUNC('month', issue_date) AS month,
        COUNT(*) AS contract_issued
    FROM contract_basic
    GROUP BY DATE_TRUNC('month', issue_date)
),
expired AS (
    SELECT
        DATE_TRUNC('month', expiry_date) AS month,
        COUNT(*) AS contract_expired
    FROM contract_basic
    WHERE expiry_date IS NOT NULL
    GROUP BY DATE_TRUNC('month', expiry_date)
)
SELECT
    COALESCE(issued.month, expired.month) AS month,
    COALESCE(contract_issued, 0) AS contract_issued,
    COALESCE(contract_expired, 0) AS contract_expired
FROM issued
FULL OUTER JOIN expired
ON issued.month = expired.month
ORDER BY month;
```

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet

# 데이터 로드
data = pd.read_csv('contract_trends.csv')  # SQL 쿼리 결과 파일
data['month'] = pd.to_datetime(data['month'])

# 계약 발생 및 종료 시각화
plt.figure(figsize=(12, 6))
plt.plot(data['month'], data['contract_issued'], label='Contracts Issued', marker='o')
plt.plot(data['month'], data['contract_expired'], label='Contracts Expired', marker='x')
plt.title('Contract Trends Over Time')
plt.xlabel('Month')
plt.ylabel('Number of Contracts')
plt.legend()
plt.grid()
plt.show()

# Prophet 모델 적용 (계약 발생 예측)
issued = data[['month', 'contract_issued']].rename(columns={'month': 'ds', 'contract_issued': 'y'})
model = Prophet()
model.fit(issued)

# 미래 데이터 프레임 생성 및 예측
future = model.make_future_dataframe(periods=12, freq='M')
forecast = model.predict(future)

# 예측 시각화
fig = model.plot(forecast)
plt.title('Forecast of Contracts Issued')
plt.show()


결과 해석
* 시각화 결과: 월별 계약 발생 및 종료 추세를 명확히 확인할 수 있습니다.
* 예측 결과: Prophet 모델을 통해 향후 12개월 동안 계약 발생 건수를 예측.
* 응용 가능성:
    * 비즈니스 전략: 계약 증가/감소 시기에 따른 마케팅 및 운영 계획 조정.
    * 리소스 할당: 계약 종료 증가 시점에 리소스를 집중적으로 배치.

# 1. 계절성 분석
계약이 특정 시점(예: 분기, 연말)에 집중되는지 확인하여 마케팅 및 운영 전략에 활용합니다.

```sql
-- 분기별 계약 발생 건수
SELECT
    CONCAT(YEAR(issue_date), ' Q', QUARTER(issue_date)) AS quarter,
    COUNT(*) AS contract_issued
FROM contract_basic
GROUP BY YEAR(issue_date), QUARTER(issue_date)
ORDER BY quarter;

-- 월별 계약 발생 평균
SELECT
    MONTH(issue_date) AS month,
    COUNT(*) AS contract_issued
FROM contract_basic
GROUP BY MONTH(issue_date)
ORDER BY month;
```

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 데이터 로드
data = pd.read_csv('contract_trends.csv')
data['month'] = pd.to_datetime(data['month'])
data['quarter'] = data['month'].dt.to_period('Q')

# 분기별 계약 건수 집계
quarterly_data = data.groupby('quarter')['contract_issued'].sum()

# 시각화
plt.figure(figsize=(10, 5))
quarterly_data.plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Contracts Issued by Quarter')
plt.ylabel('Number of Contracts')
plt.xlabel('Quarter')
plt.grid(axis='y')
plt.show()


# 2. 지역별 분석
지역별 계약 발생 및 종료를 분석하여 시장 성과를 평가합니다.

```sql
-- 지역별 계약 발생 건수
SELECT
    region_code,
    COUNT(*) AS contract_issued
FROM contract_basic
GROUP BY region_code
ORDER BY contract_issued DESC;
```

In [ ]:
region_data = data.groupby('org_code')['contract_issued'].sum()

# 시각화
plt.figure(figsize=(12, 6))
region_data.sort_values().plot(kind='barh', color='orange', edgecolor='black')
plt.title('Contracts Issued by Region')
plt.xlabel('Number of Contracts')
plt.ylabel('Region')
plt.grid(axis='x')
plt.show()

# 3. 고객 세분화와 행동 분석
가입자 행동 패턴(대면/CM/TM)을 분석하여 사용자 선호도를 파악합니다.

```sql
-- 채널별 계약 건수
SELECT
    insurance_type_code, -- 예: 대면, CM, TM
    COUNT(*) AS contract_issued
FROM contract_basic
GROUP BY insurance_type_code
ORDER BY contract_issued DESC;
```

In [ ]:
channel_data = data.groupby('insurance_type_code')['contract_issued'].sum()

# 시각화
plt.figure(figsize=(8, 4))
channel_data.plot(kind='pie', autopct='%1.1f%%', startangle=90, colors=['#ff9999','#66b3ff','#99ff99'])
plt.title('Contracts by Channel')
plt.ylabel('')
plt.show()

# 4. 예측 분석
계약 건수의 향후 추세를 예측하여 전략적 의사결정을 지원합니다.

In [ ]:
from prophet import Prophet

# Prophet 모델 초기화
model = Prophet()
issued_data = data[['month', 'contract_issued']].rename(columns={'month': 'ds', 'contract_issued': 'y'})
model.fit(issued_data)

# 예측 및 시각화
future = model.make_future_dataframe(periods=12, freq='M')
forecast = model.predict(future)
model.plot(forecast)
plt.title('Forecast of Contract Issued')
plt.show()


해석: 예측 결과를 기반으로 향후 계약 발생 증가 시기에 대비한 리소스 배분 계획을 수립할 수 있습니다.